## Introduction and Background

### Taylor series

The intuition buit on neural network(NN) solver for differential equations comes from the connection between functions and their derivatives. Generally, the derivative of a function, if it exists, represents the strength of connections between different hidden neurons if expressed by NN. Therefore, if the function can be decomposed by NN, in terms of its gradient(wrt ode, pde), the discretization will approximate the function based on information given in differential equations.

According to the universal approximation theorem, feed-forward NN can approximate any continuous function (at least) with a single hidden layer. Then it reminds us to combine NN with Taylor expansion to express the value of a function at a given point $x_0$.

Assume that $u(\textbf{x}) \in C^1(\Omega)$, where $\Omega \in R^n, n \geq 1$ is the bounded domain for $\textbf{x}$. Then we can write:
$$\begin{aligned} 
 u(\textbf{x}) &= u(\textbf{x}_0) + Du(\textbf{x}_0) \cdot (\textbf{x} - \textbf{x}_0) + \epsilon \\
       &= u(\textbf{x}_0) + \left(\frac{\partial u}{\partial \textbf{x}} |_{\textbf{x} = \text{x}_0}\right)^T \cdot (\textbf{x} - \textbf{x}_0) + \epsilon \\
       &= u(\textbf{x}_0) + \sum_{i = 1}^n \frac{\partial u(\textbf{x}_0)}{\partial x_i} \cdot (x_i - x_{0_{i}}) + \epsilon
\end{aligned}$$
where $\epsilon$ is the truncated taylor expansion residue of $u(\textbf{x})$.

So we can see that for $n$ dimensional scalar-valued functions, we can build a single layer NN based on matrix operation form to absorb expanded items in the above equations and measure the error $\epsilon$. Then we write it as :
$$\begin{aligned}
  N(\textbf{x}; w) = W_2 \sigma (W_1 \textbf{x} + \textbf{b}_1) + \textbf{b}_2 \quad \quad \quad \quad \quad \quad \quad (*)
\end{aligned}$$

where $W_1, W_2$ are weight matrices, $\textbf{b}_1, \textbf{b}_2$ are bias vectors, and $\sigma$ is a given activator. Let $w = (W_1, W_2, \textbf{b}_1, \textbf{b}_2)$ for our $N(\textbf{x}, w)$. 

And it is clear that for the above scalar $u(\textbf{x})$, $(*)$ will take all $n$ scalar inputs in $R^n, n \geq 1$ and transform into an scalar for $N(\textbf{x}; w)$. 

Intuitively, parameter pairs in $w$ can be optimized by minimizing the loss in $L^2$ norm: 
$$\begin{aligned}
 L(w) =  \lVert u(\textbf{x}) - N(\textbf{x}; w) \rVert _2  \quad \textbf{x} \in \Omega
\end{aligned}$$
Without loss of generality, we assume $\Omega \subseteq R^1$:
$$\begin{aligned}
 L(w) &=  \lVert u(\textbf{x}) - N(\textbf{x}; w) \rVert _2, \quad \textbf{x} = x \in [a, b] = \Omega \\
 &= \left(\int_a^b [u(x) - N(x; w)]^2 dx \right)^{\frac{1}{2}} \\
 &= \left(\sum_i [u(x_i) - N(x_i; w)]^2 \right)^{\frac{1}{2}}
\end{aligned}$$
where {$x_i$} is the set of discretized training points over the interval $[a, b]$.

Naturally, if the loss $L(w)$ above is under given tolerance, then $N(x; w)$ well approximates the target function:
$$ N(x; w) \approx u(x)
$$

How do Taylor series work in approximating functions combined with $(*)$?

Recall: Let $u(x) \in C^{N+1}[a,b]$ and $x_0 \in [a,b] \subseteq R^1$, then for all $x \in (a,b)$ there exists a number $c = c(x)$ that lies between $x_0$ and $x$ such that

$$ u(x) = T_N(x) + R_N(x)$$

where $T_N(x)$ is the Taylor polynomial approximation

$$T_N(x) = \sum^N_{n=0} \frac{f^{(n)}(x_0)\cdot(x-x_0)^n}{n!}$$

and $R_N(x)$ is the residual (the part of the series we left off)

$$R_N(x) = \frac{f^{(n+1)}(c) \cdot (x - x_0)^{n+1}}{(n+1)!} = \epsilon$$
This in some degree shows a fixed relation between the target function and taylor expansion coefficients. 

But, for the same variable $x$, we can modify the relations between coefficients of powers of $x$. 

For example, $u(x) = e^x$ with $x_0 = 0$:
$$\begin{aligned}
 e^{x} &= \sum^N_{n=0} e^0 \frac{x^n}{n!} = 1 + x + \frac{1}{2!} x^2 + \frac{1}{3!} x^3 + \cdots \\
 e^{2x} &= \sum^N_{n=0} e^0 \frac{(2x)^n}{n!} = 1 + 2x + \frac{2^2}{2!} x^2 + \frac{2^3}{3!} x^3 + \cdots
\end{aligned}$$
The expasion relations are fixed for each power of $x$, but 
$$ e^{x} + e^{2x} = 2 + 3x + \frac{1 + 2^2}{2!} x^2 + \frac{1 + 2^3}{3!} x^3 + \cdots
$$
The relations for powers are modified as corresponding coefficients are changed. 

What are the coefficients here for $(*)$? The weights pairs: $w$.

Now, we consider to construct the model in terms of the following general differential equation:
$$
 G\left(\vec{\! \textbf{x}}, \nabla u(\vec{\! \textbf{x}}), \nabla^2 u(\vec{\! \textbf{x}}), \cdots, \nabla^m u(\vec{\! \textbf{x}})\right) = 0 \quad \quad \quad(1)
$$
where $\textbf{x} \in \Omega , \Omega ~~\text{bounded}, ~\Omega \subseteq R^n, m, n \geq 1$

For illustration, we first consider the NN model for lower (one) dimensional ODE and rewrite $(1)$ as:
$$\begin{aligned} 
 u'(x) &= F(x, u(x)) \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad (2)\\
 u(x_0)&= u_0 \quad \quad \quad \quad \quad \quad \quad \quad \text{in} ~\Omega \subset R^1
\end{aligned}$$
And we find that if we plug in $(*)$ directly, the given initial condition cannot be satisfied, since the y-axis value will fluctuate: 
$$ N(x_0; w) \ne u_0
$$
So we force the initial condition in $(2)$ to get:
$$ \overline{u}(x; w) = u_0 + (x - x_0) \cdot N(x; w) = \tilde{N} (x; w)\quad \quad \quad (3)
$$
and it is clear that $\overline{u} = u_0$ always holds for any $w$.

Now we and plug in $(3)$ and rewrite $(2)$ as:
$$ \overline{u}'(x; w) \approx F(\overline{u}(x; w), x) = \tilde{N}' (x; w)
$$
$\Rightarrow$
$$\begin{aligned}
\overline{u}' &= \frac{d[u_0 + (x - x_0) \cdot N(x; w)]}{dt} \\
&= 0 + \frac{d(x - x_0)}{dx} \cdot N(x; w) + (x - x_0) \cdot \frac{d (N(x; w))}{dt} \\
&= N(x; w) + (x - x_0) \cdot N'(x; w) \quad \quad \quad \quad \quad  \quad (4)
\end{aligned}$$ 

Similarly, we can obtain optimal parameter pairs in $w$ by minimizing the loss in $L^2$ norm: 
$$\begin{aligned}
 L(w) &=  \lVert \overline{u}'(x; w) - \tilde{N}'(x; w) \rVert _2  \quad  x \in \Omega \\
 &\approx \sum_i [\overline{u}'(x_i; w) - F(\overline{u}(x_i; w), x_i) ]^2
\end{aligned}$$
where {$x_i$} is the set of discretized training points over the interval $[x_0, x_n] \in \Omega$.

Also if the loss $L(w)$ above is under given tolerance, then $\tilde{N}(x; w)$ may well approximate the analytical solution in $(2)$:
$$ \overline{u}(x; w) \approx u(x)_{true}
$$

### Exact form

The above implementation reminds us that through the construction in $(3)$, the NN should take into acount of all boundary conditions(BC) to express information given in differential equations. 

Basically, this can be achieved by truncating the Taylor expansion of the target function at fixed boundaries to satisfy all BC and absorb all unknown parameters independent of BC in the NN. Then the solution is inspired by classical method of trial solution for solving differetial equations.

So for $\vec{\! \textbf{x}} \in R^n$, we write our trial solution $u_t(\vec{\! \textbf{x}})$ as:
$$ u_t(\vec{\! \textbf{x}}) = A(\vec{\! \textbf{x}}) + F(\vec{\! \textbf{x}}, N(\vec{\! \textbf{x}}, w)) \quad \quad \quad \quad \quad \quad \quad (5)
$$
where $N(\vec{\! \textbf{x}}, w)$ is the single-output NN defined in $(*)$ above, $A(\vec{\! \textbf{x}})$ contains all BC, and $F(\vec{\! \textbf{x}}, N)$ contains information independent of BC and deals with minimization problems for optimal $w$.

(Remark: we only discuss Dirichlet BC in our model)

In arbitrarily complex cases of BC and higher dimensions, the definition for $(5)$ becomes difficult compared to $(4)$. So we have to force some requirements for $F$ and introduce a distance measurement indicator to construct the term $F$:
$$ F(\vec{\! \textbf{x}}, N) = N \cdot M_D(\vec{\! \textbf{x}})
$$

Definition: the function $M_D$ is an indicator of distance from the boundary in any form of $(1)$ such that:
$$F(\vec{\! \textbf{x}}, N) = 
\begin{cases}
N \cdot M_D(\vec{\! \textbf{x}})& \textbf{x} \in \Omega \setminus \partial \Omega\\
0& \textbf{x} \in \partial \Omega
\end{cases}$$

Then the model of NN can be trained in the domain through $N(\vec{\! \textbf{x}}, w)$. As we can see, the the indicator $M_D$ doesn't critically have to be the same, it may be distance functions in any form depending on expression, for example, perpendicular distance from $\textbf{x}$ to positions on the boundary will work for most convex and non-convex cases.

### Implementation of the method

For scalar functions, the NN defines a mapping $R^n \rightarrow R^1$. 

Apart from the examples given $(2)$, we show several other NN form in differential equations.

1) Single second order ODE with initial value problem (IVP)
$$\begin{aligned} 
 u''(x) &= F\left(x, u'(x), u(x)\right) \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad (6)\\
 \text{with} \quad u(x_0)&= u_0, \quad u'(x_0) = u'_0\quad \text{in} ~\Omega \subset R^1
\end{aligned}$$
The trial solution $u_t(x)$ then becomes:
$$\begin{aligned}
 u_t(x) &= A(x) + F(x, N(x, w)) \\
 &= \underbrace{u_0 + \frac{u'_0}{1!} \cdot (x - x_0)}_{A(x)} + \underbrace{\sum^N_{n=2} \frac{f^{(n)}(x_0)\cdot(x-x_0)^n}{n!} + \epsilon}_{F(x, N(x, w))} \\
 &= u_0 + u'_0 \cdot (x - x_0) + (x - x_0)^2 \cdot N(x; w) \qquad \qquad \qquad (7)
\end{aligned}$$
And optimal parameter pairs in $w$ is presented in the following equation:
$$\begin{aligned}
\text{min}\  L(w) &=  \lVert u''(x) - \tilde N(x; w) \rVert _2, \quad x \in \Omega \\
 &=\lVert u''(x) - F\left(x, u'(x), u(x)\right) \rVert _2 \\
 &= \left \{\sum_i [\frac{d^2u(x_i)}{dx^2} - F\left(x_i, u'(x_i), u(x_i)\right)]^2 \right \}^{\frac{1}{2}} \qquad \qquad (8)
\end{aligned}$$
and
$$ w = \text{arg min} \ L(w)
$$

2) Single second order ODE with boudary value problem (BVP)
$$\begin{aligned} 
 u''(x) &= F\left(x, u'(x), u(x)\right) \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad \quad (9)\\
 \text{with} \quad u(x_0)&= u_0, \quad u(x_N) = u_1\quad \text{in} ~[x_0, x_N] = \Omega \subset R^1
\end{aligned}$$
The trial solution $u_t(x)$ then becomes:
$$\begin{aligned}
 u_t(x) &= A(x) + F(x, N(x, w)) \\
 &= \begin{Bmatrix} u_0,  & x = x_0 \\ u_N, & x = x_N \end{Bmatrix} + N(x; w) \cdot M_D(x_0, x_N, x) \\
 &= u_0(x_N - x) + u_N (x - x_0) + (x - x_0)(x_N - x) \cdot N(x; w) \qquad (10)
\end{aligned}$$

And optimal parameter pairs in $w$ is presented in the following equation:
$$\begin{aligned}
\text{min}\  L(w) &=  \lVert u''(x) - \tilde N(x; w) \rVert _2, \quad x \in \Omega \\
 &= \left \{\sum_i [\frac{d^2u(x_i)}{dx^2} - F\left(x_i, u'(x_i), u(x_i)\right)]^2 \right \}^{\frac{1}{2}} \qquad \qquad (11)
\end{aligned}$$
and
$$ w = \text{arg min} \ L(w)
$$
which is the same as in 1).

3) Single second order PDE with BVP in $R^2$
For illustration, we show for Poisson equation in $R^2$:
$$\begin{aligned}
- \Delta u(x, y) = f(x, y) \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad (12)
\end{aligned}$$
with BC
$$\begin{cases}
 u(x_0, y) &= \ f_0(y) \\
 u(x_N, y) &= \ f_1(y) \\
 u(x, y_0) &= \ g_0(x) \\
 u(x, y_N) &= \ g_1(x) 
\end{cases}$$
in $x, y \in [x_0, x_N] \times [y_0, y_N]$

The trial solution $u_t(x)$ then becomes:
$$\begin{aligned}
 u_t(x, y) &= A(x, y) + F(\textbf{x}, N(\textbf{x}, w)) \\
 &= A(x, y) + N(\textbf{x}; w) \cdot M_D(x_0, x_N, y_0, y_N, x, y) \\
 &= A(x, y) + (x - x_0)(x_N - x)(y - y_0)(y_N - y)\cdot N(\textbf{x}; w)
\end{aligned}$$
where $A(x, y)$ is by definition,
$$\begin{aligned} A(x, y) &= (x_N - x) f_0(y) + (x - x_0) f_1(y) \\
&+ (y_N - y)\{g_0(x) - [(x_N -x)g_0(x_0) + (x - x_0) g_0(x_1)]\} \\
&+ (y - y_0)\{g_1(x) - [(x_N - x)g_1(0) + (x - x_0)g_1(x_N)]\}
\end{aligned}$$
And optimal parameter pairs in $w$ is presented in the following equation:
$$\begin{aligned}
\text{min}\  L(w) &=  \lVert -\Delta u(\textbf{x}) - \tilde N(\textbf{x}; w) \rVert _2, \quad x \in \Omega \\
 &= \left \{\sum_i [-\Delta u(x_i, y_i) - f\left(x_i, y_i \right)]^2 \right \}^{\frac{1}{2}} \qquad \qquad \qquad (13)
\end{aligned}$$
for $x_i, y_i \in [x_0, x_N] \times [y_0, y_N]$
and
$$ w = \text{arg min} \ L(w)
$$